<a href="https://colab.research.google.com/github/gtbook/robotics/blob/main/S65_driving_planning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -q -U gtbook


In [87]:
# no imports (yet)


# Planning for Autonomous Driving.

> Motion primitives provide a computationally efficient tool for fast, local motion planning.

<img src="Figures6/S65-Autonomous_Vehicle_with_LIDAR_and_cameras-02.jpg" alt="Splash image with steampunk autonomous car" width="40%" align=center style="vertical-align:middle;margin:10px 0px">

In previous chapters, we have mainly considered two kinds of planning problems.
For the trash sorting robot, vacuum cleaning robot, and warehouse robot, we focused
on the problem of making the best decisions in the presence of uncertainty.
In these problems, we used probability theory to quantify uncertainty,
and developed policies to maximize the expected benefit (or to minimize the expected cost)
of executing actions in a given state.
In contrast, for the differential drive robot (DDR), we considered the purely geometric
problem of planning collision-free paths. Even though this problem did not consider uncertainty,
the computational complexity of the problem precludes exact, complete solutions for all
but the simplest problems, leading to the introduction of sampling-based methods.

A common characteristic of all planning methods described to this point
is that each addresses a global problem.
For MDPs, we used value or policy iteration to establish a policy over the entire state space.
For DDRs, we searched the entire configuration space for a collision-free path.
Furthermore, the methods we developed for both problems were completely general.
Our probabilistic approaches work for arbitrary probability distributions, reward functions,
and system dynamics.
Our geometric approaches to path planning work for arbitrary environments,
and can easily be extended to robots with complex dynamics (e.g., we will extend RRTs to
the case of drones in the next chapter).

Methods that address global problems in broad generality often require significant computational
Resources (both processing power and computation time).
This can render such methods ineffective for situations in which real-time adaptivity
is required over short time horizons.
These conditions are exactly those confronted by self-driving cars,
and for this reason, in this chapter we introduce a new approach,
one that exploits motion primitives for motion planning.

## Motion Primitives

To this point, we have considered two approaches for describing motion.
For all of our probabilistic methods, we used a discrete time formulation and considered
the effects of executing an action (e.g., move forward, move left) for a small duration of time, $\Delta t$.
To plan collision-free paths, we considered artificial potential fields and RRTs, both of which
use short straight-line paths in the configuration space to connect configurations (small gradient descent
steps for potential fields, and steering toward $q_\mathrm{rand}$ for RRTs).
In each case, the language of path segments is very simple, and in each case,
a full plan will consist of many sequential steps.

This approach can be very inefficient for planning trajectories that have well-defined
characteristics.
For example, consider a car traveling in reverse that wishes to suddenly change its orientation
by completing a rapid 180-degree turn (a favorite maneuver for drivers like James Bond and Steve McQueen).
This maneuver can be achieved by a predefined
sequence of steps: after achieving a reasonable speed, remove your foot from the gas pedal;
turn left sharply and hit the breaks; at the perfect moment, release the breaks
and straighten the wheel.
When stunt drivers execute this maneuver, they do not plan step-by-step what to do.
Rather, they have pre-compiled this sequence of steps into a basic action that can be executed
with little reasoning.  This is the basic idea of **motion primitives**.


Motion primitives can be defined in numerous ways.
We could specify a geometric curve without consideration of time or dynamics
(e.g., for a parallel parking robot, we might define an
initial curve to move the car from the street into an empty parking spot).
In cases where dynamics are significant (e.g., in drone flight), we might specify
a feedback control law to be executed from an initial state until some final state is achieved.
We might parameterize these primitives by duration, by geometric properties (e.g., angle, distance),
or by state feedback conditions.
This idea is illustrated in the figure below, which shows four motion primitives
for a car.
The primitive $P_1$ corresponds to driving forward, while motion primitives $P_2$, $P_3$, and $P_4$ correspond to veering
to the left at increasingly sharp angles.


<figure id="fig:MotionPrimitives">
<img src="https://github.com/gtbook/robotics/blob/main/Figures6/motion-primitives.png?raw=1" style="width:18cm" alt="">
<figcaption>Four motion primitives for a car veering to its left. </figcaption>
</figure>

The figure above illustrates four fixed motion primitives, but it would not be difficult to generalize each of these
to a class of motions by using parametric descriptions. Below, we will see how to do this by using **polynomial trajectories** which generalize to **splines**.

## Planning using Motion Primitives

The use of motion primitives can greatly reduce the cost of planning, since the set 
of actions available at any moment in time is small and easily computed.
For the car example above, if we assume a symmetric set of motion primitives for veering to the right,
motion planning can be reduced to choosing from this set of seven possible actions at each moment in time.
If, for example, there is a slow-moving car just ahead, it might be advantageous to change lanes using one of
$P_2$, $P_3$, or $P_4$.
If there is a rapidly approaching oncoming car, it might be best to use $P_2$, to delay changing lanes
until that car has passed by.

More generally, a motion primitive typically includes a set of conditions that define when
the primitive is applicable, and a set of possible transitions to other motion primitives.
For example, it would be reasonable to veer left slightly and then drive straight, but it would
not be reasonable to transition from forward motion to reverse motion without some intermediate
maneuvering.

Under these conditions, planning can be effected by a generate-and-test approach.
At each moment in time, the planner considers the current situation, enumerates the valid
motion primitives (using preconditions for execution and set of valid transitions), and evaluates
the benefit of each admissible candidate motion primitive. This approach can be effective for
problems such as highway driving, where local context is all that is necessary for making decisions.
For example, the traffic in rural Georgia is irrelevant when leaving downtown Atlanta on
a trip to Boston.
In this case, immediate driving decisions depend on the car just ahead, and the nearby
cars in adjacent lanes.


##  Polynomial Trajectories

Let’s begin with the simple problem of changing lanes along a straight stretch of highway. The situation is illustrated in the figure below.

<figure id="fig:LaneChange">
<img src="https://github.com/gtbook/robotics/blob/main/Figures6/lane-change.png?raw=1" style="width:18cm" alt="">
<figcaption>Initial and final configuration for a lane change maneuver. </figcaption>
</figure>

Here, we have taken the $x$-axis to be longitudinal direction (parallel to the highway), and the $y$-axis is along the lateral direction (perpendicular to the highway).  For now, let us assume that the lane change maneuver begins at $x=0$, $y=0$, and that the desired ending configuration is $ x = x_\mathrm{g}, y = y_\mathrm{g}$, i.e., at the end of the maneuver (at $ x = x_\mathrm{g}$), 
the car will have reached the center of the left lane (i.e., $ y = y_\mathrm{g}$).

At first glance, it might seem that this could be accomplished by a simple linear trajectory of the form

$$ y(x) = \frac{ y_\mathrm{g}}{x_\mathrm{g} } x $$

At the start of the maneuver, $x=0$, which matches the initial condition $y(0)=0$, and at $ x = x_\mathrm{g}$
we match the end condition $y(x_\mathrm{g}) = y_\mathrm{g}$.
This trajectory is illustrated in the figure below.

<figure id="fig:LinearLaneChange">
<img src="https://github.com/gtbook/robotics/blob/main/Figures6/linear-lane-change.png?raw=1" style="width:18cm" alt="">
<figcaption>Linear trajectory that attempts to achieve a lane change. </figcaption>
</figure>

Note that this trajectory requires an instantaneous change in the car’s heading, from 0 at $x = 0^-$ to
$\frac{ y_\mathrm{g}}{x_\mathrm{g} }$ at $x=0^+$. Cars, of course, are not capable of executing maneuvers with velocity discontinuities (under normal driving conditions), so this trajectory could not be executed by a normal car.
However, it is not difficult to generalize this approach to solve this problem.

Consider the $d^{th}$ order polynomial 

$$ y(x) = \sum_{i=0}^d \alpha_i x^i $$

For $nd=1$, we obtain the linear trajectory above by setting
$\alpha_0 = 0, \alpha_1 = \frac{ y_\mathrm{g}}{x_\mathrm{g} }$.
In other words, if we begin with the general linear trajectory
$y(x) = \alpha_0 + \alpha_1 x$, we can solve for $\alpha_0$ and $\alpha_1$ to match the initial
and final values for the lateral position $y$.
This shouldn’t be surprising; the linear polynomial has two free parameters ($\alpha_0$ and $\alpha_1$),
so it makes sense that we could satisfy two independent constraints (initial and final values for $y$) by
an appropriate choice for these parameters.

In general, the parameters of an $n^{th}$ order polynomial can be chosen to satisfy $n+1$ independent constraints.
So, if we wish to match initial and final conditions on heading (which is defined by the first derivative of $y$),
we would require a cubic polynomial, and if we wished to also satisfy lateral acceleration constraints, we would need a fifth order polynomial.
The lateral velocity and acceleration for the trajectory are given by the first and second derivatives of $y$,
which we denote by $y’$ and $y’’$, respectively.
For a fifth order polynomial, we have 


$$ 
\begin{aligned}
y(x) &=& \alpha_0 + \alpha_1 x + \alpha_2 x^2 + \alpha_3 x^3 + \alpha_4 x^4 + \alpha_5 x^5\\
y’(x) &=& \alpha_1 + 2 \alpha_2 x + 3 \alpha_3 x^2 + 4 \alpha_4 x^3 + 5 \alpha_5 x^4\\
y’’(x) &=&  2 \alpha_2  +6 \alpha_3  x + 12 \alpha_4 x^2 + 20 \alpha_5 x^3 
\end{aligned}
$$

For a simple lane change, it is typically desirable to have zero lateral velocity and zero lateral acceleration at the initial and final positions. This leads to the following system of equations:

$$ 
\begin{aligned}
y(0) = 0& =& \alpha_0 \\
y’(0) = 0 & =& \alpha_1 \\
y’’(0) = 0 &=&   2 \alpha_2  \\
y(x_\mathrm{g}) = y_\mathrm{g} &=&
     \alpha_0 + \alpha_1 x_\mathrm{g} + \alpha_2 x_\mathrm{g}^2 + \alpha_3 x_\mathrm{g}^3 + \alpha_4 x_\mathrm{g}^4 + \alpha_5 x_\mathrm{g}^5 \\
y’(x_\mathrm{g}) = 0 &=&
   \alpha_1 + 2 \alpha_2 x_\mathrm{g} + 3 \alpha_3 x_\mathrm{g}^2 + 4 \alpha_4 x_\mathrm{g}^3 + 5 \alpha_5 x_\mathrm{g}^4 \\
y’’(x_\mathrm{g}) =  0 &=&
  2 \alpha_2  +6 \alpha_3  x_\mathrm{g} + 12 \alpha_4 x_\mathrm{g}^2 + 20 \alpha_5 x_\mathrm{g}^3 
\end{aligned}
$$



Note that these six equations are all linear in the parameters $\alpha_i$, so it is a simple matter to solve
these.

While the derivation above produced a single polynomial trajectory,
it is a simple matter to extend this formalism to construct trajectories
that are composed of multiple consecutive polynomial segments.
In fact, we have actually done exactly this in the above derviation,
if we consider that for $x < 0$ and for $x  > x_\mathrm{g}$ the trajectory $y(Xt)$ is linear and pararallel to the $x$-axis,
i.e., we have solved for a special case of three polynomial segments.

Suppose that we wish to construct a piecewise polynomial trajectory that
exactly reaches certain via points, $(x_i, y_i)$, i.e., we are given the
sequence of couples $(x_0,y_0) \dots (x_n, y_n)$,
and we want to ensure that the lateral position of the car is given by $y_i$ at longitudinal
position $x_i$.
We can build the trajectory $y$ from a sequence of polynomials $P_k$ as follows

$$ 
\begin{align}
y(x) &= P_k(x)  \;\;\;\;\;\ x_k \leq x < x_{k+1},\;\; 0 \leq k < n\\
P_k(x)  &= \sum_{i=0}^{d} \alpha_{ki} (x-x_k)^i \\
\end{align}
$$

The coefficients $\alpha_{ki}$ are chosen to ensure the desired continuity and smoothness properties.
For example, if we want to ensure that position, velocity, and accelerations are continuous
throughout the trajectory, we must enforce

$$
\begin{align}
y_k = &P_{k-1}(x_k) = P_{k}(x_k), \;\;\; k = 1, \dots n-1 \\
&P'_{k-1}(x_k) = P'_{k}(x_k), \;\;\; k = 1, \dots n-1 \\
&P''_{k-1}(x_k) = P''_{k}(x_k), \;\;\; k = 1, \dots n-1 
\end{align}
$$

Note that the first set of constraints also ensure that $y(x_k) = y_k$ for each transition point.
In order to satisfy the boundary conditions for the start and end of the trajectory, we
must enforce

$$
\begin{align}
y_0 = &P_{0}(x_0)  \\
y_n = &P_{n}(x_n)  \\
v_0 = &P'_{0}(x_0)  \\
v_n = &P'_{n}(x_n)  \\
a_0 = &P''_{0}(x_0)  \\
a_n = &P''_{n}(x_n)  \\
\end{align}
$$

where $v_k$ and $a_k$ denote the velocity and acceleration, respectively,
at position $x_k$.

Note that in this formulation, the accelerations and velocities at the junctions of two
adjacent splines are not specified; the velocity and acceleration must match for the two
polynomial segments, but the specific values of velocity and acceleration are not specified.
Because of this, we have fewer constraints, and therefore require fewer parameters
$\alpha_{ki}$.
In particular, if we merely want to ensure continuity of position, velocity, and acceleration
at the junctions, cubic polynomials are sufficient.
This is different from the situation above, in which we used a quintic polynomial to achieve
specific values of velocity and acceleration (all of which were set to zero) at the junctions
of the two linear segments with the lane-changing trajectory.
You should count the constraints, and convince yourself that this is true.

## Splines

The polynomial trajectores that we defined above were from a subset
of the more general class of **splines**.
In general, a spline is a continuous, piecewise polynomial curve, and we are not
necessarily given the specific values for the transition points between adjacent
segments.
Parametric splines can be defined as